# Result of Proposed Dataset

In [ ]:
# Downloading the dataset
import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Checking the availablility of GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  

# Defining the path of dataset 
dataset_path = "/home/Balanced Tomato Layered Augmentation"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Checking if dataset exists or not
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("balanced-tomato-dataset-sfqxc")
    version = project.version(2)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Loading the YOLO model
model = YOLO("yolov8n")  



Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Balanced-Tomato-Dataset-2 in yolov8:: 100%|█████████████████████████████████████████| 4062/4062 [00:01<00:00, 2276.66it/s]


In [ ]:
import os
from collections import defaultdict

dataset_path = "/home/Balanced Tomato Layered Augmentation"

# Defining splits and corresponding label directories
splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}

# Tracking image filenames (unique) per split
split_image_counts = defaultdict(set)

# Tracking image filenames per class per split
split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}

# Processing each split
for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)

# Output counts
print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(5):  # Classes 0 to 4
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")

# Combining and finding total unique images
total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/Balanced Tomato Layered Augmentation/train/labels
Scanning: /home/Balanced Tomato Layered Augmentation/valid/labels
Scanning: /home/Balanced Tomato Layered Augmentation/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 861 images
  Class 1: 122 images
  Class 2: 504 images
  Class 3: 396 images
  Class 4: 0 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images
  Class 4: 0 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images
  Class 4: 0 images

Total number of images in each split:
  Train: 1883 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 2025


In [ ]:
# Training 
model.train(
    data=data_yaml,
    name="TR",
    device=device_ids,  # Using all the available GPUs
    epochs=100, 
    batch=256,  
    workers=16,  # Using multiple data loading workers
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/Balanced Tomato Layered Augmentation/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=TR4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_h

train: Scanning /home/Balanced Tomato Layered Augmentation/train/labels... 1883 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1883/1883 [00:02<00:00, 644.89it/s]


train: New cache created: /home/Balanced Tomato Layered Augmentation/train/labels.cache


val: Scanning /home/Balanced Tomato Layered Augmentation/valid/labels... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<00:00, 1400.62it/s]


val: New cache created: /home/Balanced Tomato Layered Augmentation/valid/labels.cache
Plotting labels to runs/detect/TR4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/TR4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89        0.6      0.782       0.69      0.673

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100       8.1G      0.169     0.1414     0.7916         23        640: 100%|██████████| 8/8 [00:02<00:00,  3.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.37it/s]


                   all         89         89      0.611      0.783      0.682      0.665

100 epochs completed in 0.087 hours.
Optimizer stripped from runs/detect/TR4/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/TR4/weights/best.pt, 6.3MB

Validating runs/detect/TR4/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.67it/s]


                   all         89         89      0.789      0.777      0.826      0.804
                     H         59         59       0.88      0.932      0.966      0.947
                     K          7          7          1      0.766      0.995      0.955
                     N         12         12      0.595        0.5      0.411      0.402
                   N_K         11         11       0.68      0.909       0.93       0.91
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/TR4


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/Balanced Tomato Layered Augmentation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]


                   all         89         89      0.789      0.777      0.826      0.803
                     H         59         59      0.881      0.932      0.966      0.947
                     K          7          7          1      0.768      0.995      0.955
                     N         12         12      0.595        0.5      0.411      0.402
                   N_K         11         11      0.681      0.909       0.93       0.91
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/TR42
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f88934e1c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.0040

In [5]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/Balanced Tomato Layered Augmentation/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5

val: New cache created: /home/Balanced Tomato Layered Augmentation/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


                   all         53         53      0.938      0.848      0.914      0.888
                     H         32         32      0.911          1      0.974      0.956
                     K          7          7      0.873          1       0.96      0.949
                     N          8          8          1      0.392      0.725      0.674
                   N_K          6          6      0.967          1      0.995      0.973
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/TR43
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f89cd76ccd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,   

# Results of Original Tomato dataset(356 images)

In [ ]:
# Downloading the dataset
import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow


num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  


dataset_path = "/home/Original"
data_yaml = os.path.join(dataset_path, "data.yaml")


if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("og_tomato-7lgrg")
    version = project.version(3)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")


model = YOLO("yolov8n")  




Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to OG_Tomato-3 in yolov8:: 100%|█████████████████████████████████████████████████████████| 724/724 [00:00<00:00, 3707.24it/s]


In [ ]:
import os
from collections import defaultdict

dataset_path = "/home/Original"


splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}


split_image_counts = defaultdict(set)


split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}


for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)


print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(5):  # Classes 0 to 4
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")

# Combine all images to find total unique images
total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/Original/train/labels
Scanning: /home/Original/valid/labels
Scanning: /home/Original/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 145 images
  Class 1: 19 images
  Class 2: 27 images
  Class 3: 23 images
  Class 4: 0 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images
  Class 4: 0 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images
  Class 4: 0 images

Total number of images in each split:
  Train: 214 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 356


In [ ]:
# Training
model.train(
    data=data_yaml,
    name="org",
    device=device_ids,  
    epochs=100, 
    batch=256,  
    workers=16,  
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/Original/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=org, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=

train: Scanning /home/Original/train/labels... 214 images, 0 backgrounds, 0 corrupt: 100%|██████████| 214/214 [00:00<00:00, 921.44it/s]


train: New cache created: /home/Original/train/labels.cache


val: Scanning /home/Original/valid/labels... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<00:00, 753.67it/s]


val: New cache created: /home/Original/valid/labels.cache
Plotting labels to runs/detect/org/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/org
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89      0.684      0.601      0.688      0.657

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      5.98G     0.2416      1.201     0.7965         54        640: 100%|██████████| 1/1 [00:00<00:00,  3.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


                   all         89         89      0.641      0.578      0.687      0.658

100 epochs completed in 0.027 hours.
Optimizer stripped from runs/detect/org/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/org/weights/best.pt, 6.3MB

Validating runs/detect/org/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]


                   all         89         89      0.643      0.578      0.689      0.659
                     H         59         59      0.847      0.898      0.928      0.917
                     K          7          7      0.584      0.429      0.562      0.553
                     N         12         12      0.342      0.167      0.391       0.37
                   N_K         11         11        0.8      0.818      0.875      0.795
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/org


In [9]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/Original/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


                   all         89         89       0.64      0.578      0.687      0.656
                     H         59         59      0.847      0.898      0.928      0.918
                     K          7          7      0.583      0.429       0.56      0.551
                     N         12         12      0.341      0.167      0.387      0.361
                   N_K         11         11      0.791      0.818      0.875      0.795
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/org2
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f88937aa1f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.0040

In [10]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/Original/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<00:00, 582.30it/

val: New cache created: /home/Original/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


                   all         53         53      0.713      0.614      0.611      0.588
                     H         32         32      0.818          1      0.918      0.895
                     K          7          7      0.658      0.429      0.603      0.601
                     N          8          8          1      0.363      0.544      0.491
                   N_K          6          6      0.378      0.667      0.377      0.364
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/org3
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f88780d0a00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,   

# Results of dataset containing images with only Flip augmentation applied

In [ ]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow


num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  


dataset_path = "/home/only flip"
data_yaml = os.path.join(dataset_path, "data.yaml")


if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("only_train-z8q6q")
    version = project.version(1)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")


model = YOLO("yolov8n")  




Dataset already exists. Using the existing dataset.


In [ ]:
import os
from collections import defaultdict

splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}

split_image_counts = defaultdict(set)

split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}


for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)


print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(4):  # Classes 0 to 4
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")

total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/only flip/train/labels
Scanning: /home/only flip/valid/labels
Scanning: /home/only flip/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 246 images
  Class 1: 35 images
  Class 2: 45 images
  Class 3: 40 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images

Total number of images in each split:
  Train: 366 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 508


In [ ]:
# Training
model.train(
    data=data_yaml,
    name="only flip",
    device=device_ids,  
    epochs=100, 
    batch=256,  
    workers=16,  
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/only flip/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=only flip2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=No

train: Scanning /home/only flip/train/labels.cache... 366 images, 0 backgrounds, 0 corrupt: 100%|██████████| 366/366 [00:00<?, ?it/s]
val: Scanning /home/only flip/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]


Plotting labels to runs/detect/only flip2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/only flip2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      7.76G     0.8335      3.462      1.209         61        640: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]
  0%|          | 0/2 [00:00<?, ?it/s]

                   all         89         89     0.0051          1      0.264      0.235

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89      0.521      0.739      0.685      0.665

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.07G      0.214     0.5819     0.7961         28        640: 100%|██████████| 2/2 [00:00<00:00,  3.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


                   all         89         89      0.524      0.737      0.682      0.659

100 epochs completed in 0.034 hours.
Optimizer stripped from runs/detect/only flip2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/only flip2/weights/best.pt, 6.3MB

Validating runs/detect/only flip2/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.20it/s]


                   all         89         89      0.806      0.692      0.765      0.741
                     H         59         59      0.886       0.92      0.911      0.896
                     K          7          7      0.844      0.857      0.808      0.789
                     N         12         12      0.679      0.355      0.492      0.468
                   N_K         11         11      0.816      0.636      0.848      0.812
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/only flip2


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/only flip/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.84s/it]


                   all         89         89      0.806      0.692      0.766      0.745
                     H         59         59      0.886      0.919      0.911      0.896
                     K          7          7      0.846      0.857      0.808      0.789
                     N         12         12       0.68      0.356      0.495      0.471
                   N_K         11         11      0.815      0.636      0.848      0.822
Speed: 0.9ms preprocess, 2.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/only flip22
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa6567b5130>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,   

In [5]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/only flip/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<00:00, 675.06it

val: New cache created: /home/only flip/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


                   all         53         53      0.587      0.753      0.676      0.609
                     H         32         32       0.89          1       0.97      0.953
                     K          7          7      0.412      0.857      0.761      0.577
                     N          8          8      0.509      0.375      0.482      0.418
                   N_K          6          6      0.538      0.781      0.493       0.49
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/only flip23
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa657a9b8b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004

# Results of dataset containing images with Flip and Rotation augmentations applied

In [ ]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow


num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  


dataset_path = "/home/flip rotation"
data_yaml = os.path.join(dataset_path, "data.yaml")


if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("flip-rotation-90-zlhxm")
    version = project.version(1)
    dataset = version.download("yolov8")         
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")


model = YOLO("yolov8n")  




Dataset already exists. Using the existing dataset.


In [ ]:
import os
from collections import defaultdict

splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}

split_image_counts = defaultdict(set)

split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}


for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)


print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(4): 
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")


total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/flip rotation/train/labels
Scanning: /home/flip rotation/valid/labels
Scanning: /home/flip rotation/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 492 images
  Class 1: 70 images
  Class 2: 90 images
  Class 3: 80 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images

Total number of images in each split:
  Train: 732 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 874


In [ ]:
# Training
model.train(
    data=data_yaml,
    name="flip rotation",
    device=device_ids,  
    epochs=100, 
    batch=256,  
    workers=16,  
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/flip rotation/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=flip rotation2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False,

train: Scanning /home/flip rotation/train/labels.cache... 732 images, 0 backgrounds, 0 corrupt: 100%|██████████| 732/732 [00:00<?, ?it/s]
val: Scanning /home/flip rotation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]


Plotting labels to runs/detect/flip rotation2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/flip rotation2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      7.76G     0.7532      3.494      1.145        118        640: 100%|██████████| 3/3 [00:02<00:00,  1.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]
  0%|          | 0/3 [00:00<?, ?it/s]

                   all         89         89    0.00494          1      0.243      0.216

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89      0.452      0.733      0.619      0.603

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.07G     0.2113     0.3546     0.8063         55        640: 100%|██████████| 3/3 [00:00<00:00,  3.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.23it/s]


                   all         89         89       0.45      0.781      0.616        0.6

100 epochs completed in 0.045 hours.
Optimizer stripped from runs/detect/flip rotation2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/flip rotation2/weights/best.pt, 6.3MB

Validating runs/detect/flip rotation2/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]Exception in thread Thread-2:
Traceback (most recent call last):
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do_one_step()
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 32, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/mansi/anaconda3/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "/home/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 496, in rebuild_storage_fd
Exception in thread Thread-2:
Traceback (most recent 

                   all         89         89      0.742      0.637      0.749      0.719
                     H         59         59      0.888      0.537       0.92      0.882
                     K          7          7      0.525      0.857      0.879      0.842
                     N         12         12      0.969       0.25      0.521      0.503
                   N_K         11         11      0.587      0.903      0.675      0.648
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/flip rotation2


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/flip rotation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.84s/it]


                   all         89         89      0.742      0.637      0.748      0.718
                     H         59         59      0.888      0.537      0.918      0.881
                     K          7          7      0.526      0.857      0.878      0.841
                     N         12         12      0.968       0.25      0.521      0.503
                   N_K         11         11      0.586      0.902      0.675      0.648
Speed: 0.7ms preprocess, 2.0ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/flip rotation22
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f339c220f40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003

In [5]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/flip rotation/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<00:00, 663.

val: New cache created: /home/flip rotation/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


                   all         53         53      0.558      0.794      0.761      0.722
                     H         32         32      0.886      0.969      0.977      0.957
                     K          7          7      0.294          1      0.784      0.734
                     N          8          8      0.548      0.375      0.603      0.538
                   N_K          6          6      0.503      0.833      0.678       0.66
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/flip rotation23
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f339c220a90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0

# # Results of dataset containing images with only Flip, Rotation and Saturation augmentations applied

In [ ]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  

dataset_path = "/home/flip rotation saturation"
data_yaml = os.path.join(dataset_path, "data.yaml")

if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("flip-rotation-90-saturation-w9mwt")
    version = project.version(1)
    dataset = version.download("yolov8")       
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")


model = YOLO("yolov8n") 



Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to flip,-rotation(90)-&-saturation-1 in yolov8:: 100%|█████████████████████████████████| 2594/2594 [00:00<00:00, 4973.70it/s]


In [ ]:
import os
from collections import defaultdict

splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}

split_image_counts = defaultdict(set)

split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}

for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)


print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(4): 
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")


total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/flip rotation saturation/train/labels
Scanning: /home/flip rotation saturation/valid/labels
Scanning: /home/flip rotation saturation/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 872 images
  Class 1: 122 images
  Class 2: 162 images
  Class 3: 138 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images

Total number of images in each split:
  Train: 1294 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 1436


In [ ]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="flip rotation saturation",
    device=device_ids,  
    epochs=100, 
    batch=256,  
    workers=16,  
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/flip rotation saturation/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=flip rotation saturation, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=Fals

train: Scanning /home/flip rotation saturation/train/labels... 1294 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1294/1294 [00:01<00:00, 699.34it/s]


train: New cache created: /home/flip rotation saturation/train/labels.cache


val: Scanning /home/flip rotation saturation/valid/labels... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<00:00, 989.63it/s]


val: New cache created: /home/flip rotation saturation/valid/labels.cache
Plotting labels to runs/detect/flip rotation saturation/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/flip rotation saturation
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89      0.464      0.793      0.688      0.672

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.11G     0.2096     0.2063     0.7752          4        640: 100%|██████████| 6/6 [00:01<00:00,  3.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.44it/s]


                   all         89         89      0.475      0.802      0.709      0.692

100 epochs completed in 0.067 hours.
Optimizer stripped from runs/detect/flip rotation saturation/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/flip rotation saturation/weights/best.pt, 6.3MB

Validating runs/detect/flip rotation saturation/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]Exception in thread Thread-2:
Traceback (most recent call last):
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do_one_step()
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 32, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/mansi/anaconda3/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "/homelib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 496, in rebuild_storage_fd
    fd = df.detach()
  File "/home/mansi/anaconda3/lib

                   all         89         89       0.55      0.795      0.806       0.79
                     H         59         59      0.884      0.905      0.912      0.908
                     K          7          7      0.232      0.857      0.877      0.857
                     N         12         12      0.662      0.417      0.482      0.465
                   N_K         11         11      0.423          1      0.952      0.929
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/flip rotation saturation


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/flip rotation saturation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.82s/it]


                   all         89         89      0.552      0.794      0.806       0.79
                     H         59         59      0.884      0.904      0.912      0.908
                     K          7          7      0.231      0.857      0.877      0.857
                     N         12         12      0.668      0.417      0.482      0.465
                   N_K         11         11      0.424          1      0.952      0.931
Speed: 0.7ms preprocess, 1.9ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/flip rotation saturation2
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f13b01c3970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,  

In [5]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/flip rotation saturation/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<

val: New cache created: /home/flip rotation saturation/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


                   all         53         53      0.625      0.769      0.762      0.728
                     H         32         32      0.938      0.945      0.973      0.947
                     K          7          7      0.333      0.998      0.846      0.808
                     N          8          8       0.76       0.25      0.581      0.529
                   N_K          6          6      0.468      0.881       0.65       0.63
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/flip rotation saturation3
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f13b9737a60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.00

# All augmentationss at once (Standard Augmentation)

In [ ]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  


dataset_path = "/home/standard augmentation"
data_yaml = os.path.join(dataset_path, "data.yaml")

if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="C04lllSQbIxhR8ypzRYd")
    project = rf.workspace("tomato-nutrition").project("for-making-2x-rbi1b")
    version = project.version(2)
    dataset = version.download("yolov8")                  
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

model = YOLO("yolov8n")  




Dataset already exists. Using the existing dataset.


In [ ]:
import os
from collections import defaultdict


splits = {
    "train": os.path.join(dataset_path, "train", "labels"),
    "valid": os.path.join(dataset_path, "valid", "labels"),
    "test": os.path.join(dataset_path, "test", "labels"),
}


split_image_counts = defaultdict(set)

split_class_to_images = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set),
}


for split_name, labels_dir in splits.items():
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        split_image_counts[split_name].add(label_file)

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                split_class_to_images[split_name][class_id].add(label_file)


print("\nNumber of images per class in each split:")
for split_name in splits:
    print(f"\n{split_name.capitalize()} Set:")
    for class_id in range(4):  # Classes 0 to 4
        count = len(split_class_to_images[split_name][class_id])
        print(f"  Class {class_id}: {count} images")

print("\nTotal number of images in each split:")
for split_name in splits:
    count = len(split_image_counts[split_name])
    print(f"  {split_name.capitalize()}: {count} images")


total_images = set()
for images in split_image_counts.values():
    total_images.update(images)

print(f"\nTotal number of unique images in the dataset: {len(total_images)}")


Scanning: /home/standard augmentation/train/labels
Scanning: /home/standard augmentation/valid/labels
Scanning: /home/standard augmentation/test/labels

Number of images per class in each split:

Train Set:
  Class 0: 1450 images
  Class 1: 190 images
  Class 2: 270 images
  Class 3: 230 images

Valid Set:
  Class 0: 59 images
  Class 1: 7 images
  Class 2: 12 images
  Class 3: 11 images

Test Set:
  Class 0: 32 images
  Class 1: 7 images
  Class 2: 8 images
  Class 3: 6 images

Total number of images in each split:
  Train: 2140 images
  Valid: 89 images
  Test: 53 images

Total number of unique images in the dataset: 2282


In [ ]:
# Training
model.train(
    data=data_yaml,
    name="standard augmentation",
    device=device_ids,  
    epochs=100, 
    batch=256,  
    workers=16,  
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/standard augmentation/data.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=standard augmentation2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, sa

train: Scanning /home/standard augmentation/train/labels.cache... 2140 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2140/2140 [00:00<?, ?it/s]
val: Scanning /home/standard augmentation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]


Plotting labels to runs/detect/standard augmentation2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.002), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/standard augmentation2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      7.76G     0.7806      3.477      1.174         52        640: 100%|██████████| 9/9 [00:04<00:00,  1.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


                   all         89         89    0.00904          1      0.265      0.256

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         89         89      0.731      0.727      0.762      0.747

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.07G      0.204      0.151     0.7943         23        640: 100%|██████████| 9/9 [00:02<00:00,  3.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.39it/s]


                   all         89         89       0.67       0.73      0.755      0.743

100 epochs completed in 0.095 hours.
Optimizer stripped from runs/detect/standard augmentation2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/standard augmentation2/weights/best.pt, 6.3MB

Validating runs/detect/standard augmentation2/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]Exception in thread Thread-2:
Traceback (most recent call last):
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/mansi/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do_one_step()
  File "/home/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 32, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/mansi/anaconda3/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "/home/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 496, in rebuild_storage_fd
    fd = df.detach()
  File "/home/mansi/anaconda3/li

                   all         89         89      0.801      0.741      0.812      0.798
                     H         59         59      0.881      0.881      0.926       0.92
                     K          7          7      0.843      0.857      0.964      0.938
                     N         12         12      0.556      0.315      0.394       0.38
                   N_K         11         11      0.923      0.909      0.963      0.955
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/standard augmentation2


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/standard augmentation/valid/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]


                   all         89         89      0.801      0.741      0.812      0.799
                     H         59         59      0.881       0.88      0.928      0.921
                     K          7          7      0.843      0.857      0.964      0.938
                     N         12         12      0.557      0.316      0.394       0.38
                   N_K         11         11      0.924      0.909      0.963      0.955
Speed: 0.7ms preprocess, 1.9ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/standard augmentation22
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb1c922d0a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    

In [5]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)

Running testing...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/standard augmentation/test/labels... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<00:

val: New cache created: /home/standard augmentation/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]


                   all         53         53      0.826      0.786      0.809      0.792
                     H         32         32      0.938       0.94      0.956      0.946
                     K          7          7          1      0.996      0.995      0.983
                     N          8          8      0.586      0.375      0.434      0.419
                   N_K          6          6      0.781      0.833      0.852       0.82
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/standard augmentation23
Test Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb1ca7138b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.0030